# MNIST Machine Learning Case Study

## A Comprehensive Guide to Machine Learning Fundamentals

This notebook provides a detailed exploration of essential machine
learning concepts using the classic MNIST handwritten digit dataset.
We’ll combine theoretical explanations with practical Python
implementations to build strong intuition and practical skills.

### Table of Contents

1.  [Introduction to the MNIST Dataset](#1-introduction)
2.  [Data Exploration and Visualization](#2-exploration)
3.  [Supervised Learning Fundamentals](#3-supervised-learning)
4.  [Feature Engineering for Image Data](#4-feature-engineering)
5.  [Linear Algebra and Probability
    Foundations](#5-linear-algebra-probability)
6.  [Understanding Covariance and Correlation](#6-covariance)
7.  [Principal Component Analysis (PCA)](#7-pca)
8.  [Data Distribution Analysis](#8-distributions)
9.  [Cross-Validation Techniques](#9-cross-validation)
10. [Bias-Variance Tradeoff](#10-bias-variance)
11. [Overfitting and Underfitting](#11-overfitting-underfitting)
12. [Regularization Methods](#12-regularization)
13. [Classical Machine Learning Models](#13-classical-models)
14. [Model Comparison and Analysis](#14-comparison)
15. [Conclusion and Best Practices](#15-conclusion)

## 1. Introduction to the MNIST Dataset <a id="1-introduction"></a>

The MNIST dataset is one of the most widely used benchmark datasets in
machine learning. It consists of 28×28 grayscale images of handwritten
digits (0-9).

Let’s begin by loading the dataset and understanding its structure.

``` python
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics, preprocessing, model_selection
import time
import warnings
warnings.filterwarnings('ignore')

# Set plotting aesthetics
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style="whitegrid")
```

``` python
# Load the MNIST dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

# Display the first few rows
train_data.head()
```

### Dataset Structure

-   Each image is 28×28 pixels, flattened into 784 features
-   Pixel values range from 0 (white) to 255 (black)
-   The training set includes 60,000 examples
-   The test set includes 10,000 examples
-   The first column in the training set contains the label (0-9)

## 2. Data Exploration and Visualization <a id="2-exploration"></a>

Let’s explore and visualize the dataset to better understand the data
we’re working with.

``` python
# Basic statistics of the training data
train_data.describe()
```

``` python
# Class distribution - how many examples of each digit?
plt.figure(figsize=(10, 6))
class_distribution = train_data['label'].value_counts().sort_index()
ax = sns.barplot(x=class_distribution.index, y=class_distribution.values)
plt.title('Distribution of Digits in the Training Set', fontsize=14)
plt.xlabel('Digit', fontsize=12)
plt.ylabel('Count', fontsize=12)
for i, v in enumerate(class_distribution.values):
    ax.text(i, v + 100, str(v), ha='center')
plt.show()

print("Class distribution:")
for digit, count in class_distribution.items():
    print(f"Digit {digit}: {count} samples ({count/len(train_data)*100:.2f}%)")
```

Now, let’s visualize some examples of the digits:

``` python
# Function to plot a digit
def plot_digit(data, index):
    digit = data.iloc[index, 1:].values.reshape(28, 28)
    plt.imshow(digit, cmap='gray_r')
    plt.title(f"Label: {data.iloc[index, 0]}")
    plt.axis('off')

# Function to plot multiple digits
def plot_multiple_digits(data, nrows=5, ncols=5):
    fig, axes = plt.subplots(nrows, ncols, figsize=(12, 12))
    axes = axes.flatten()
    
    # Get random indices
    random_indices = np.random.randint(0, len(data), nrows * ncols)
    
    for i, ax in enumerate(axes):
        idx = random_indices[i]
        digit = data.iloc[idx, 1:].values.reshape(28, 28)
        ax.imshow(digit, cmap='gray_r')
        ax.set_title(f"Label: {data.iloc[idx, 0]}")
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Plot random sample of digits
plot_multiple_digits(train_data)
```

``` python
# Let's analyze the pixel intensity distribution
plt.figure(figsize=(12, 6))
# Get all pixel values and create a histogram
pixel_values = train_data.iloc[:, 1:].values.flatten()
plt.hist(pixel_values, bins=50, color='blue', alpha=0.7)
plt.title('Distribution of Pixel Intensities Across All Images', fontsize=14)
plt.xlabel('Pixel Value (0-255)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

print(f"Mean pixel value: {pixel_values.mean():.2f}")
print(f"Standard deviation: {pixel_values.std():.2f}")
print(f"Min pixel value: {pixel_values.min()}")
print(f"Max pixel value: {pixel_values.max()}")
print(f"Percentage of pixels with zero value: {(pixel_values == 0).mean() * 100:.2f}%")
```

``` python
# Visualize average digit for each class
plt.figure(figsize=(15, 8))

for i in range(10):
    # Select all examples with the current digit
    digit_data = train_data[train_data['label'] == i].iloc[:, 1:].values
    
    # Calculate the average pixel values
    avg_digit = digit_data.mean(axis=0).reshape(28, 28)
    
    plt.subplot(2, 5, i + 1)
    plt.imshow(avg_digit, cmap='gray_r')
    plt.title(f"Average of Digit: {i}")
    plt.axis('off')

plt.tight_layout()
plt.show()
```

## 3. Supervised Learning Fundamentals <a id="3-supervised-learning"></a>

MNIST classification is a quintessential supervised learning problem: -
We have labeled examples (digits 0-9) - We want to learn a mapping from
pixel features to digit labels - We will train on known examples and
predict on unknown examples

### Key Elements of Supervised Learning

``` python
# Basic approach to supervised learning with MNIST

# 1. Separate features (X) and labels (y)
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values

# 2. Create a validation set
# We'll use 20% of the training data as a validation set
X_train, X_val, y_train, y_val = model_selection.train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Training set size: {X_train.shape[0]} examples")
print(f"Validation set size: {X_val.shape[0]} examples")

# 3. Normalize the features (scale pixel values to [0, 1])
X_train = X_train / 255.0
X_val = X_val / 255.0

# 4. Define evaluation metrics for classification
def evaluate_model(model, X, y_true, name="Model"):
    """Evaluate model performance with multiple metrics."""
    # Make predictions
    y_pred = model.predict(X)
    
    # Calculate metrics
    accuracy = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred, average='weighted')
    precision = metrics.precision_score(y_true, y_pred, average='weighted')
    recall = metrics.recall_score(y_true, y_pred, average='weighted')
    
    print(f"{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")
    print(f"Precision (weighted): {precision:.4f}")
    print(f"Recall (weighted): {recall:.4f}")
    
    # Confusion matrix
    plt.figure(figsize=(10, 8))
    cm = metrics.confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}', fontsize=14)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.show()
    
    return accuracy, f1, precision, recall
```

### Theory of Supervised Learning

Supervised learning is about finding a function (f) that maps inputs (X)
to outputs (y): \[ y = f(X) + \]

This function is learned from labeled examples by minimizing a loss
function that measures the difference between predictions and true
labels.

For classification problems like MNIST, we use: - **Loss functions**:
Cross-entropy, hinge loss - **Evaluation metrics**: Accuracy, precision,
recall, F1-score - **Learning algorithms**: Decision trees, neural
networks, SVMs, etc.

## 4. Feature Engineering for Image Data <a id="4-feature-engineering"></a>

While deep learning often works with raw pixels, traditional machine
learning benefits significantly from feature engineering. Let’s explore
some techniques:

``` python
# Original features: raw pixel values
print(f"Original feature space dimension: {X_train.shape[1]}")

# Some basic feature engineering approaches:

# 1. HISTOGRAM OF ORIENTED GRADIENTS (HOG)
from skimage.feature import hog
from skimage import exposure

def get_hog_features(X, visualize=False):
    # Take a small sample to demonstrate
    sample_digit = X[0].reshape(28, 28)
    
    # Extract HOG features
    hog_features, hog_image = hog(
        sample_digit, 
        orientations=9, 
        pixels_per_cell=(4, 4),
        cells_per_block=(2, 2), 
        visualize=True,
        block_norm='L2-Hys'
    )
    
    if visualize:
        # Rescale HOG image for better visualization
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        
        # Plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.imshow(sample_digit, cmap='gray_r')
        ax1.set_title('Original Image')
        ax1.axis('off')
        
        ax2.imshow(hog_image_rescaled, cmap='gray_r')
        ax2.set_title('HOG Features')
        ax2.axis('off')
        plt.show()
    
    return hog_features

# Demonstrate HOG features
hog_features = get_hog_features(X_train, visualize=True)
print(f"HOG feature space dimension: {len(hog_features)}")
```

``` python
# 2. ZONING - Divide image into zones and calculate statistics
def get_zoning_features(X, n_zones=4, visualize=False):
    # Reshape for easier zoning
    sample_digit = X[0].reshape(28, 28)
    
    # Define zone size
    zone_height = 28 // n_zones
    zone_width = 28 // n_zones
    
    # Create features based on zone statistics
    zoning_features = []
    
    if visualize:
        plt.figure(figsize=(10, 10))
        plt.imshow(sample_digit, cmap='gray_r')
        
    for i in range(n_zones):
        for j in range(n_zones):
            # Extract zone
            zone = sample_digit[i*zone_height:(i+1)*zone_height, 
                               j*zone_width:(j+1)*zone_width]
            
            # Calculate statistics
            zone_mean = zone.mean()
            zone_std = zone.std()
            zone_max = zone.max()
            
            # Add features
            zoning_features.extend([zone_mean, zone_std, zone_max])
            
            if visualize:
                # Draw zone boundaries
                plt.plot([j*zone_width, j*zone_width], 
                         [i*zone_height, (i+1)*zone_height], 'r-')
                plt.plot([(j+1)*zone_width, (j+1)*zone_width], 
                         [i*zone_height, (i+1)*zone_height], 'r-')
                plt.plot([j*zone_width, (j+1)*zone_width], 
                         [i*zone_height, i*zone_height], 'r-')
                plt.plot([j*zone_width, (j+1)*zone_width], 
                         [(i+1)*zone_height, (i+1)*zone_height], 'r-')
                
                # Add text for zone statistics
                plt.text(j*zone_width + zone_width/2, i*zone_height + zone_height/2, 
                         f"{zone_mean:.1f}", color='blue',
                         horizontalalignment='center', verticalalignment='center')
    
    if visualize:
        plt.title('Zoning with Mean Values', fontsize=14)
        plt.axis('off')
        plt.show()
    
    return np.array(zoning_features)

# Demonstrate zoning features
zoning_features = get_zoning_features(X_train, visualize=True)
print(f"Zoning feature space dimension: {len(zoning_features)}")
```

``` python
# 3. HISTOGRAM FEATURES
def get_histogram_features(X, n_bins=10, visualize=False):
    # Take sample digit
    sample_digit = X[0].reshape(28, 28)
    
    # Calculate histogram
    hist, bin_edges = np.histogram(sample_digit, bins=n_bins, range=(0, 1))
    hist = hist / np.sum(hist)  # Normalize
    
    if visualize:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot original image
        ax1.imshow(sample_digit, cmap='gray_r')
        ax1.set_title('Original Image')
        ax1.axis('off')
        
        # Plot histogram
        ax2.bar(range(n_bins), hist, width=0.8, 
                align='center', color='blue', alpha=0.7)
        ax2.set_xlabel('Pixel Intensity Bin')
        ax2.set_ylabel('Normalized Frequency')
        ax2.set_title('Histogram Features')
        ax2.set_xticks(range(n_bins))
        ax2.set_xticklabels([f"{edge:.1f}" for edge in bin_edges[:-1]])
        plt.tight_layout()
        plt.show()
    
    return hist

# Demonstrate histogram features
hist_features = get_histogram_features(X_train, visualize=True)
print(f"Histogram feature space dimension: {len(hist_features)}")
```

### Applying Feature Engineering to the Dataset

Let’s implement a simple feature extraction pipeline to demonstrate how
we could prepare data for traditional models:

``` python
# Function to extract features from multiple images
def extract_features(X, n_samples=None, n_zones=4, n_bins=10):
    """Extract multiple types of features from images."""
    if n_samples is None:
        n_samples = len(X)
    else:
        n_samples = min(n_samples, len(X))
    
    # Sample selection
    indices = np.random.choice(len(X), n_samples, replace=False)
    X_sample = X[indices]
    
    # Initialize feature array
    all_features = []
    
    for img in X_sample:
        img_reshaped = img.reshape(28, 28)
        
        # Get zoning features
        zone_features = []
        zone_height = 28 // n_zones
        zone_width = 28 // n_zones
        
        for i in range(n_zones):
            for j in range(n_zones):
                zone = img_reshaped[i*zone_height:(i+1)*zone_height, 
                                   j*zone_width:(j+1)*zone_width]
                zone_mean = zone.mean()
                zone_std = zone.std()
                zone_max = zone.max()
                zone_features.extend([zone_mean, zone_std, zone_max])
        
        # Get histogram features
        hist, _ = np.histogram(img, bins=n_bins, range=(0, 1))
        hist = hist / np.sum(hist)  # Normalize
        
        # Get simple edge features (difference between adjacent pixels)
        edges_h = np.abs(np.diff(img_reshaped, axis=0))
        edges_v = np.abs(np.diff(img_reshaped, axis=1))
        edge_features = [edges_h.mean(), edges_h.std(), edges_v.mean(), edges_v.std()]
        
        # Combine all features
        features = np.concatenate([zone_features, hist, edge_features])
        all_features.append(features)
    
    return np.array(all_features)

# Extract features from a small sample for demonstration
X_train_features = extract_features(X_train, n_samples=5000)
X_val_features = extract_features(X_val, n_samples=1000)

print(f"Engineered feature space dimension: {X_train_features.shape[1]}")
```

## 5. Linear Algebra and Probability Foundations <a id="5-linear-algebra-probability"></a>

Machine learning algorithms leverage linear algebra and probability
theory extensively. Let’s explore key concepts using our MNIST data.

### Linear Algebra Concepts with MNIST

``` python
# 1. Matrix operations with images

# Select single digit
digit = X_train[0].reshape(28, 28)

# Matrix operations
# Transpose - Flip along diagonal
digit_transposed = digit.T

# Add noise to create a related but different image
noise = np.random.normal(0, 0.1, size=digit.shape)
digit_noisy = np.clip(digit + noise, 0, 1)

# Matrix multiplication (convolution-like operation)
# Create a simple edge detection kernel
edge_kernel = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])

# Apply kernel via correlation (similar to convolution)
from scipy import signal
digit_edges = signal.correlate2d(digit, edge_kernel, mode='same')

# Visualize
fig, axs = plt.subplots(1, 4, figsize=(16, 4))
axs[0].imshow(digit, cmap='gray_r')
axs[0].set_title('Original')
axs[0].axis('off')

axs[1].imshow(digit_transposed, cmap='gray_r')
axs[1].set_title('Transposed')
axs[1].axis('off')

axs[2].imshow(digit_noisy, cmap='gray_r')
axs[2].set_title('With Noise')
axs[2].axis('off')

axs[3].imshow(digit_edges, cmap='gray_r')
axs[3].set_title('Edge Detection')
axs[3].axis('off')

plt.tight_layout()
plt.show()
```

``` python
# 2. Vector spaces and distances
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

# Select two digits for comparison
digit1 = X_train[0]  # First image
digit2 = X_train[1]  # Second image
digit3 = X_train[np.where(y_train == y_train[0])[0][1]]  # Another image of same class as digit1

# Calculate distances
euclidean_1_2 = euclidean_distances([digit1], [digit2])[0][0]
euclidean_1_3 = euclidean_distances([digit1], [digit3])[0][0]

cosine_1_2 = cosine_similarity([digit1], [digit2])[0][0]
cosine_1_3 = cosine_similarity([digit1], [digit3])[0][0]

# Visualize
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(digit1.reshape(28, 28), cmap='gray_r')
axs[0].set_title(f'Digit 1 (Class: {y_train[0]})')
axs[0].axis('off')

axs[1].imshow(digit2.reshape(28, 28), cmap='gray_r')
axs[1].set_title(f'Digit 2 (Class: {y_train[1]})')
axs[1].axis('off')

axs[2].imshow(digit3.reshape(28, 28), cmap='gray_r')
axs[2].set_title(f'Digit 3 (Class: {y_train[np.where(y_train == y_train[0])[0][1]]})')
axs[2].axis('off')

plt.tight_layout()
plt.show()

print("Distance metrics:")
print(f"Euclidean distance between Digit 1 and Digit 2 (different classes): {euclidean_1_2:.4f}")
print(f"Euclidean distance between Digit 1 and Digit 3 (same class): {euclidean_1_3:.4f}")
print()
print(f"Cosine similarity between Digit 1 and Digit 2 (different classes): {cosine_1_2:.4f}")
print(f"Cosine similarity between Digit 1 and Digit 3 (same class): {cosine_1_3:.4f}")
```

### Probability Concepts with MNIST

``` python
# 1. Class probabilities
class_probs = train_data['label'].value_counts(normalize=True).sort_index()

plt.figure(figsize=(10, 6))
ax = sns.barplot(x=class_probs.index, y=class_probs.values)
plt.title('Probability Distribution of Digits', fontsize=14)
plt.xlabel('Digit', fontsize=12)
plt.ylabel('Probability', fontsize=12)
for i, v in enumerate(class_probs):
    ax.text(i, v + 0.005, f"{v:.3f}", ha='center')
plt.show()

# Prior probabilities
print("Prior class probabilities:")
for digit, prob in class_probs.items():
    print(f"P(Digit = {digit}) = {prob:.4f}")
```

``` python
# 2. Pixel intensity distributions by class
plt.figure(figsize=(15, 10))

for i in range(10):
    # Get all samples of this digit
    digit_samples = train_data[train_data['label'] == i].iloc[:, 1:].values
    
    # Get average pixel values
    avg_intensity = digit_samples.mean(axis=0)
    
    # Plot histogram of average intensities
    plt.subplot(3, 4, i+1)
    plt.hist(avg_intensity, bins=50, alpha=0.7)
    plt.title(f'Pixel Intensity Distribution: Digit {i}')
    plt.xlabel('Average Pixel Value')
    plt.ylabel('Frequency')
    plt.xlim(0, 255)

plt.tight_layout()
plt.show()
```

``` python
# 3. Bayes' Theorem in action - Probabilistic classification
from sklearn.naive_bayes import GaussianNB

# Train a Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train[:5000], y_train[:5000])  # Use a small subset for speed

# Get probabilities for a single example
test_digit = X_val[0]
test_digit_class = y_val[0]
class_probabilities = nb_classifier.predict_proba([test_digit])[0]

# Visualize
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.imshow(test_digit.reshape(28, 28), cmap='gray_r')
plt.title(f'Test Digit (True Class: {test_digit_class})', fontsize=14)
plt.axis('off')

plt.subplot(1, 2, 2)
bars = plt.bar(range(10), class_probabilities)
plt.title('Class Probabilities (Naive Bayes)', fontsize=14)
plt.xlabel('Digit Class', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.xticks(range(10))

# Highlight the true class
bars[test_digit_class].set_color('red')

plt.tight_layout()
plt.show()

print(f"Predicted class: {nb_classifier.predict([test_digit])[0]}")
print(f"True class: {test_digit_class}")
print("\nClass probabilities:")
for i, prob in enumerate(class_probabilities):
    print(f"P(Digit = {i} | Image) = {prob:.4f}")
```

## 6. Understanding Covariance and Correlation <a id="6-covariance"></a>

Covariance measures how two variables change together, revealing
relationships between features. Let’s explore covariance in MNIST data:

``` python
# Select a small subset of pixels to analyze covariance
# Choose pixels from the middle of the image (more interesting)
central_pixels = [
    # Row indices from middle of image
    392, 393, 394, 395,  # Middle row pixels
    364, 365, 366, 367   # Pixels from row above
]

# Create a dataframe with these pixels
pixel_sample = train_data.iloc[:1000, [0] + [i+1 for i in central_pixels]]
pixel_sample.columns = ['label'] + [f'pixel_{i}' for i in central_pixels]

# Calculate covariance matrix
cov_matrix = pixel_sample.iloc[:, 1:].cov()

# Calculate correlation matrix
corr_matrix = pixel_sample.iloc[:, 1:].corr()

# Visualize covariance matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cov_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Covariance Matrix of Central Pixels', fontsize=14)
plt.tight_layout()
plt.show()

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Central Pixels', fontsize=14)
plt.tight_layout()
plt.show()
```

``` python
# Show what these pixels look like on an image
def highlight_pixels(pixel_indices, image_shape=(28, 28)):
    """Create a visualization of where selected pixels are located in the image."""
    # Create empty image
    image = np.zeros(image_shape)
    
    # Fill in the selected pixels
    for idx in pixel_indices:
        # Convert flattened index back to 2D
        row = idx // image_shape[1]
        col = idx % image_shape[1]
        image[row, col] = 1
    
    return image

# Create visualization
highlighted = highlight_pixels(central_pixels)

plt.figure(figsize=(8, 8))
plt.imshow(highlighted, cmap='gray_r')
plt.title('Location of Selected Pixels for Covariance Analysis', fontsize=14)
plt.axis('off')
plt.show()
```

``` python
# Visualize covariance across different digits
plt.figure(figsize=(15, 12))

for i in range(10):
    # Get samples for this digit
    digit_pixels = train_data[train_data['label'] == i].iloc[:200, 1:].values
    
    # Calculate covariance matrix (use only subset of pixels for visualization)
    # Take a 10x10 grid from the center
    center_start_row = 9
    center_start_col = 9
    center_pixels = []
    
    for r in range(10):
        for c in range(10):
            idx = (center_start_row + r) * 28 + (center_start_col + c)
            center_pixels.append(idx)
    
    # Extract these pixels
    digit_center = digit_pixels[:, center_pixels]
    
    # Calculate covariance
    cov = np.cov(digit_center, rowvar=False)
    
    # Plot
    plt.subplot(2, 5, i+1)
    sns.heatmap(cov, cmap='coolwarm', xticklabels=False, yticklabels=False)
    plt.title(f'Covariance: Digit {i}', fontsize=14)

plt.tight_layout()
plt.show()
```

``` python
# Interpreting covariance in classification context
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Use small subset for demonstration
X_sample = X_train[:5000]
y_sample = y_train[:5000]

# Fit LDA model
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X_sample, y_sample)

# Plot classes in LDA space
plt.figure(figsize=(12, 10))
scatter = plt.scatter(X_lda[:, 0], X_lda[:, 1], c=y_sample, 
                      cmap='tab10', alpha=0.6, s=20)
plt.colorbar(scatter, label='Digit')
plt.title('LDA Projects Data Based on Covariance Structure', fontsize=14)
plt.xlabel('LD 1', fontsize=12)
plt.ylabel('LD 2', fontsize=12)
plt.tight_layout()
plt.show()

print("LDA leverages class covariance matrices to find the most discriminative projections.")
print(f"LDA explained variance ratio: {lda.explained_variance_ratio_}")
```

## 7. Principal Component Analysis (PCA) <a id="7-pca"></a>

PCA is a crucial dimensionality reduction technique that projects data
onto directions of maximum variance.

\`\`\`python from sklearn.decomposition import PCA

# Apply P